<a href="https://colab.research.google.com/github/DaanMoura/AprendizadoDeMaquina/blob/main/MovieDataset_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
import copy

In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/DaanMoura/AprendizadoDeMaquina/main/data/movies.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def get_date_year(date_column):
  try:
    return date_column.split('-')[0]
  except:
    return -1

def get_date_month(date_column):
  try:
    return date_column.split('-')[1]
  except:
    return -1

In [4]:
dataset['release_year'] = pd.Series(list(map(get_date_year, dataset["release_date"].tolist())))
dataset['release_month'] = pd.Series(list(map(get_date_month, dataset["release_date"].tolist())))

In [5]:
dataset.drop(["adult", "belongs_to_collection", "homepage", "id", "imdb_id", "original_title", "overview", "poster_path", "tagline", "title", "video","spoken_languages","release_date"], axis=1,inplace=True)
df = dataset.copy()

In [6]:
def get_first(o):
  try:
    return o.split(',')[0].split(':')[1].strip().strip("'")
  except:
    return -1
  

In [7]:
def get_first_number(o):
  try:
    id = o.split(',')[0].split(':')[1].strip().strip("'")
    if str(id).isnumeric():
      return id
    else:
      return -1
  except:
    return -1

In [8]:
pd.set_option('display.max_colwidth', -1)
df["production_companies"] = pd.Series(list(map(get_first, df["production_companies"].tolist())))
df["production_countries"] = pd.Series(list(map(get_first, df["production_countries"].tolist())))
df["genres"] = pd.Series(list(map(get_first_number, df["genres"].tolist())))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [9]:
colunas = ["original_language","production_companies","production_countries"]

In [10]:
labels, levels = pd.factorize(df['production_companies'])

In [11]:
string_columns = ['original_language','production_companies','production_countries','status']

for column in string_columns:
  labels, levels = pd.factorize(df[column])
  df[column] = labels

In [12]:
df = df.apply(pd.to_numeric, errors='coerce')
df = df.fillna(-1)

In [13]:
#separa o alvo e os dados
Y = df["vote_average"]
for i in range(len(Y)):
  if Y[i] < 2:
    Y[i] = "Muito Ruim"
  elif Y[i] < 4:
    Y[i] = "Ruim"
  elif Y[i] < 6:
    Y[i] = "Mediano"
  elif Y[i] < 8:
    Y[i] = "Bom"
  else:
    Y[i] = "Muito Bom"

X = df.drop(["vote_average"], axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __

In [14]:
#/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg'
dataset.drop([29503],axis=0,inplace=True)


In [15]:
#separa em teste e treino
x_tr, x_te, y_tr, y_te = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [16]:
x_tr

,budget,genres,original_language,popularity,production_companies,production_countries,revenue,runtime,status,vote_count,release_year,release_month
41712,0.0,18,0,1.664214,9804,0,0.0,100.0,0,21.0,2016,3
25646,8500000.0,10751,0,0.994222,34,9,0.0,90.0,0,17.0,2014,12
29596,0.0,36,0,7.046825,20,25,1822250.0,129.0,0,259.0,2014,10
4526,0.0,18,0,2.188691,1117,0,0.0,105.0,0,3.0,1989,8
41710,0.0,10749,0,0.126559,33,9,0.0,93.0,0,2.0,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11284,0.0,18,0,0.225313,33,9,0.0,118.0,0,4.0,1983,1
44732,0.0,16,0,0.002638,33,9,0.0,6.0,0,0.0,1916,10
38158,0.0,53,0,0.000026,2082,0,0.0,74.0,0,0.0,1974,3
860,0.0,18,0,0.978989,263,0,0.0,96.0,0,4.0,1998,10


In [17]:
x_tr

,budget,genres,original_language,popularity,production_companies,production_countries,revenue,runtime,status,vote_count,release_year,release_month
41712,0.0,18,0,1.664214,9804,0,0.0,100.0,0,21.0,2016,3
25646,8500000.0,10751,0,0.994222,34,9,0.0,90.0,0,17.0,2014,12
29596,0.0,36,0,7.046825,20,25,1822250.0,129.0,0,259.0,2014,10
4526,0.0,18,0,2.188691,1117,0,0.0,105.0,0,3.0,1989,8
41710,0.0,10749,0,0.126559,33,9,0.0,93.0,0,2.0,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11284,0.0,18,0,0.225313,33,9,0.0,118.0,0,4.0,1983,1
44732,0.0,16,0,0.002638,33,9,0.0,6.0,0,0.0,1916,10
38158,0.0,53,0,0.000026,2082,0,0.0,74.0,0,0.0,1974,3
860,0.0,18,0,0.978989,263,0,0.0,96.0,0,4.0,1998,10


In [18]:
y_tr

41712    Mediano   
25646    Mediano   
29596    Bom       
4526     Bom       
41710    Muito Bom 
           ...     
11284    Bom       
44732    Muito Ruim
38158    Muito Ruim
860      Bom       
15795    Mediano   
Name: vote_average, Length: 36372, dtype: object

In [19]:
dtc = DecisionTreeClassifier()
dtc
dtc.fit(x_tr, y_tr)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [20]:
y_pred = dtc.predict(x_te)
accuracy_score(y_te, y_pred)

0.5411260171541676

Hyperparams estimators

In [25]:
param_grid = {'criterion': ['entropy', 'gini'],
              'max_depth': range(2,30,2),
              'min_samples_leaf': range(2,10,2),
              'min_impurity_decrease': np.linspace(0,0.5,10)}

In [26]:
dtc = DecisionTreeClassifier()
gs = GridSearchCV(dtc, param_grid=param_grid)
gs.fit(x_tr,y_tr)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['entropy', 'gini'],
                

In [27]:
gs.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [28]:
dtc = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
dtc.fit(x_tr, y_tr)
y_pred = dtc.predict(x_te)
accuracy_score(y_te, y_pred)

0.6224983505608094